In [1]:
import torch
import torch.nn as nn
from torch.optim import SGD
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random

# Carregando a Rede

In [2]:
class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.Matrix1 = nn.Linear(28**2,100)
        self.Matrix2 = nn.Linear(100,50)
        self.Matrix3 = nn.Linear(50,4)
        self.R = nn.ReLU()
    def forward(self,x):
        x = x.view(-1,28**2)
        x = self.R(self.Matrix1(x))
        x = self.R(self.Matrix2(x))
        x = self.Matrix3(x)
        return x.squeeze()

In [3]:
RedeNeural = MyNeuralNet()

In [4]:
RedeNeural.load_state_dict(torch.load('RedeNeural1.pth'))

C:\Users\R2\AppData\Local\Temp\ipykernel_14300\3765925349.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  RedeNeural.load_state_dict(torch.load('RedeNeural1.pth'))


<All keys matched successfully>

In [5]:
print(RedeNeural)

MyNeuralNet(
  (Matrix1): Linear(in_features=784, out_features=100, bias=True)
  (Matrix2): Linear(in_features=100, out_features=50, bias=True)
  (Matrix3): Linear(in_features=50, out_features=4, bias=True)
  (R): ReLU()
)


# Parte do desenho

In [6]:
def ImgNormalizer(img):
    w = len(img[0])
    h = len(img[1])
    DrawX = []
    DrawY = []
    for x in range(w):
        for y in range(h):
            if img[x][y] == 255:
                DrawX.append(x)
                DrawY.append(y)

    # Se não ouver desenho nenhum, retorna imagens completamente vazias
    if not DrawX:
        imgSmall = np.full((28,28), 0, dtype=np.uint8)
        imgSmall2 = np.full((28,28), 0, dtype=np.uint8)
        return imgSmall,imgSmall2
        
    # Pega os valores mínimos e máximos das coordenadas
    MaxXValue = np.max(DrawX)
    MinXValue = np.min(DrawX)
    MaxYValue = np.max(DrawY)
    MinYValue = np.min(DrawY)
    
    # Calcula as amplitudes
    XAmplitude = MaxXValue - MinXValue
    YAmplitude = MaxYValue - MinYValue
    
    if XAmplitude > YAmplitude:
        # Altera as coordenadas para ir de 0 ate a aplitude maxima e centraliza a amplitude menor
        Amp = XAmplitude
        for i in range(len(DrawX)):
            DrawX[i] = DrawX[i] - MinXValue
            DrawY[i] = DrawY[i] - MinYValue + (Amp/2) - (YAmplitude/2)
    else:
        # Altera as coordenadas para ir de 0 ate a aplitude maxima e centraliza a amplitude menor
        Amp = YAmplitude
        for i in range(len(DrawX)):
            DrawX[i] = DrawX[i] - MinXValue + (Amp/2) - (XAmplitude/2)
            DrawY[i] = DrawY[i] - MinYValue
    
    DrawSmall = []
    for i in range(len(DrawX)):
        # Reduz a amplitude para 28x28 com 2 pixels de borda e arredonda os resultados
        newpair = []
        newpair.append(((23/Amp)*DrawX[i]) + 2)
        newpair.append(((23/Amp)*DrawY[i]) + 2) 
        newpairR = np.int32(np.rint(newpair))
        DrawSmall.append(newpairR)
    # Deixa apenas os pares únicos, excluindo os repetidos após o arredondamento
    DrawSmall = np.unique(DrawSmall,axis=0)

    #Desenha a imagem 28x28
    imgSmall = np.full((28,28), 0, dtype=np.uint8)
    imgSmall2 = np.full((28,28), 0, dtype=np.uint8)
    for pair in DrawSmall:
        imgSmall[pair[0]][pair[1]] = 255
        cv2.circle(imgSmall2,(pair[1],pair[0]),1,(255,255,255),-1)

    return imgSmall,imgSmall2

In [10]:
class DrawingApp:
    def __init__(self):
        # Variáveis iniciais
        self.isDrawing = False
        self.Acertou = False
    
    def drawLine(self,event,x,y,flags,param):
        #Desenha na imagem enquanto o mouse se move após o botão esquerdo do mouse é pressionado
        #Desenha em 2 imagens ao mesmo tempo: Uma para ter uma referencia do desenho e uma para gerar as coordenadas do mouse
        img1 = param[0]
        img2 = param[1]
        if event == cv2.EVENT_LBUTTONDOWN:
            self.isDrawing = True
        elif event == cv2.EVENT_MOUSEMOVE and self.isDrawing:
            cv2.circle(img1,(x,y),5,(255,255,255),-1)
            cv2.circle(img2,(x,y),0,(255,255,255),-1)
        elif event == cv2.EVENT_LBUTTONUP:
            self.isDrawing = False

    def run(self):

        # Cria imagens base vazias
        img = np.full((640,640), 0, dtype=np.uint8)
        img2 = np.full((640,640), 0, dtype=np.uint8)

        countImg = np.full((400,400), 0, dtype=np.uint8)
        font = cv2.FONT_HERSHEY_SIMPLEX
        texto1 = "Desenhe"
        cv2.putText(countImg, texto1, (75, 75), font, 1, (255,255,255), 3, cv2.LINE_AA)
        randomnumber = random.randint(0,3)
        match randomnumber:
            case 0:
                target = "Baixo"
            case 1:
                target = "Cima"
            case 2:
                target = "Esquerda"
            case 3:
                target = "Direita"

        cv2.putText(countImg, target, (75, 100), font, 1, (255,255,255), 3, cv2.LINE_AA)


        # Cria um vetor com as duas imagens
        images = [img,img2]

        #gera o nome da janela
        windowName = 'drawing app'
        cv2.namedWindow(windowName)
        windowName2 = 'count'
        cv2.namedWindow(windowName2)

        #chama a função de desenho que chama a função drawLine com os eventos do mouse
        cv2.setMouseCallback(windowName,self.drawLine,images)

        while True:
            cv2.imshow(windowName,images[0])
            cv2.imshow(windowName2,countImg)
            wait = cv2.waitKey(1)
            if wait == ord('q'):
                # Sai do Loop da etapa de desenho
                break
            elif wait == ord('r'):
                # Limpa o conteudo da tela para que novas imagens possam ser geradas
                images[0] = np.full((640,640), 0, dtype=np.uint8)
                images[1] = np.full((640,640), 0, dtype=np.uint8)
            elif wait == ord('s'):
                # Gera as imagens na resolução 28x28
                imagesSmall = ImgNormalizer(images[1])
                
                predicition = RedeNeural(torch.from_numpy(np.asarray(imagesSmall[1]))/255).argmax()
                print(predicition)

                if predicition == randomnumber:
                    countImg = np.full((400,400), 0, dtype=np.uint8)
                    cv2.putText(countImg, texto1, (75, 75), font, 1, (255,255,255), 3, cv2.LINE_AA)
                    randomnumber = random.randint(0,3)
                    match randomnumber:
                        case 0:
                            target = "Baixo"
                        case 1:
                            target = "Cima"
                        case 2:
                            target = "Esquerda"
                        case 3:
                            target = "Direita"

                    cv2.putText(countImg, target, (75, 100), font, 1, (255,255,255), 3, cv2.LINE_AA)

                    cv2.putText(countImg, "Acertou!", (75, 125), font, 1, (255,255,255), 3, cv2.LINE_AA)
                else:
                    countImg = np.full((400,400), 0, dtype=np.uint8)
                    cv2.putText(countImg, texto1, (75, 75), font, 1, (255,255,255), 3, cv2.LINE_AA)
                    randomnumber = random.randint(0,3)
                    match randomnumber:
                        case 0:
                            target = "Baixo"
                        case 1:
                            target = "Cima"
                        case 2:
                            target = "Esquerda"
                        case 3:
                            target = "Direita"

                    cv2.putText(countImg, target, (75, 100), font, 1, (255,255,255), 3, cv2.LINE_AA)

                    cv2.putText(countImg, "Errou!", (75, 125), font, 1, (255,255,255), 3, cv2.LINE_AA)
        
        # Fecha a janela e termina processo
        cv2.destroyAllWindows()

In [8]:
def Drawing():
    app = DrawingApp()
    app.run()

In [12]:
if __name__ == '__main__':
    Drawing()

tensor(1)
tensor(2)
tensor(2)
tensor(0)
tensor(3)
tensor(3)
tensor(1)
tensor(1)
tensor(0)
tensor(0)
tensor(3)
tensor(0)
